In [1]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
import numpy as np
import jieba as jb
from pandas import *
import re

In [2]:
labels = [];datas = [];
fr = open('toutiao_cat_data_Reduced.txt',encoding='utf-8')
for line in fr.readlines():
    lineArr = line.strip().split("_!_")#strip()删除换行符，用split("_!_")把每行文本按照_!_分开
    labels.append(lineArr[2])#标签
    datas.append(lineArr[3] + lineArr[4])#特征：标题+新闻关键词
#用DataFrame()存储
df = pandas.DataFrame()
df['data'] = datas
df['label'] = labels

In [3]:
df

,data,label
0,"今年我省将抓好健康扶贫五项重点工作贫困人口,健康扶贫,贫困县,乡镇卫生院,镇巴县",news_agriculture
1,"农村大爷窖藏了12年的纯粮酒，这样的酒你觉得值多少钱？大爷,农村",news_agriculture
2,"无核早红葡萄丰产栽培技术杀菌剂,可湿性粉剂,复合肥,灰霉病,许昌市",news_agriculture
3,DNF同为buff装，金镶玉和高科技相比如何？,news_game
4,"LPL再组智囊团 MSI还有RNG的陪练团洲际赛,LPL,陪练团,MSI,智囊团",news_game
...,...,...
21657,"第一次去男友家, 男友母亲竟偷偷对我说, 赶紧逃",news_story
21658,"小孩救了一老乞丐，送给小孩一块玉石，被继母抢去后被雷劈死玉石,刘浩,继母,那块玉石",news_story
21659,"夜归宿舍遇陌生人洗脚 被发现后推说“走错了”洗脚,走错门",news_story
21660,"我结婚哥哥给我十万红包，上厕所时听见他接电话，我含泪还给了他还给你,婚礼,饭店,红包",news_story


In [4]:
#列表中添加类型id
df['id'] = df['label'].factorize()[0]#factorize创建数字表示类别变量，对每一个类别映射一个id
id_df = df[['label', 'id']].drop_duplicates().sort_values('id').reset_index(drop=True)#删除重复项、根据id排序、重置index

In [5]:
df

,data,label,id
0,"今年我省将抓好健康扶贫五项重点工作贫困人口,健康扶贫,贫困县,乡镇卫生院,镇巴县",news_agriculture,0
1,"农村大爷窖藏了12年的纯粮酒，这样的酒你觉得值多少钱？大爷,农村",news_agriculture,0
2,"无核早红葡萄丰产栽培技术杀菌剂,可湿性粉剂,复合肥,灰霉病,许昌市",news_agriculture,0
3,DNF同为buff装，金镶玉和高科技相比如何？,news_game,1
4,"LPL再组智囊团 MSI还有RNG的陪练团洲际赛,LPL,陪练团,MSI,智囊团",news_game,1
...,...,...,...
21657,"第一次去男友家, 男友母亲竟偷偷对我说, 赶紧逃",news_story,13
21658,"小孩救了一老乞丐，送给小孩一块玉石，被继母抢去后被雷劈死玉石,刘浩,继母,那块玉石",news_story,13
21659,"夜归宿舍遇陌生人洗脚 被发现后推说“走错了”洗脚,走错门",news_story,13
21660,"我结婚哥哥给我十万红包，上厕所时听见他接电话，我含泪还给了他还给你,婚礼,饭店,红包",news_story,13


#### 中文文本预处理

In [6]:
#删除标点函数
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
#读入停用词列表
def stopwordslist():  
    stopwords = [line.strip() for line in open("chineseStopWords.txt", 'r', encoding='utf-8').readlines()]  
    return stopwords  
stopwords = stopwordslist()

In [7]:
df['clean_data'] = df['data'].apply(remove_punctuation)#删除标点
df['cut_data'] = df['clean_data'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))#分词，同时过滤停用词
df

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LY\AppData\Local\Temp\jieba.cache
Loading model cost 0.814 seconds.
Prefix dict has been built successfully.


,data,label,id,clean_data,cut_data
0,"今年我省将抓好健康扶贫五项重点工作贫困人口,健康扶贫,贫困县,乡镇卫生院,镇巴县",news_agriculture,0,今年我省将抓好健康扶贫五项重点工作贫困人口健康扶贫贫困县乡镇卫生院镇巴县,今年 我省 抓好 健康 扶贫 五项 重点 工作 贫困人口 健康 扶贫 贫困县 乡镇 卫生院 镇巴县
1,"农村大爷窖藏了12年的纯粮酒，这样的酒你觉得值多少钱？大爷,农村",news_agriculture,0,农村大爷窖藏了12年的纯粮酒这样的酒你觉得值多少钱大爷农村,农村 大爷 窖藏 12 年 纯 粮酒 酒 觉得 值 钱 大爷 农村
2,"无核早红葡萄丰产栽培技术杀菌剂,可湿性粉剂,复合肥,灰霉病,许昌市",news_agriculture,0,无核早红葡萄丰产栽培技术杀菌剂可湿性粉剂复合肥灰霉病许昌市,无核 早 红葡萄 丰产 栽培技术 杀菌剂 可湿性 粉剂 复合肥 灰霉病 许昌市
3,DNF同为buff装，金镶玉和高科技相比如何？,news_game,1,DNF同为buff装金镶玉和高科技相比如何,DNF 同为 buff 装 金镶玉 高科技 相比
4,"LPL再组智囊团 MSI还有RNG的陪练团洲际赛,LPL,陪练团,MSI,智囊团",news_game,1,LPL再组智囊团MSI还有RNG的陪练团洲际赛LPL陪练团MSI智囊团,LPL 再组 智囊团 MSI RNG 陪练 团 洲际 赛 LPL 陪练 团 MSI 智囊团
...,...,...,...,...,...
21657,"第一次去男友家, 男友母亲竟偷偷对我说, 赶紧逃",news_story,13,第一次去男友家男友母亲竟偷偷对我说赶紧逃,第一次 男友 家 男友 母亲 竟 偷偷 说 赶紧 逃
21658,"小孩救了一老乞丐，送给小孩一块玉石，被继母抢去后被雷劈死玉石,刘浩,继母,那块玉石",news_story,13,小孩救了一老乞丐送给小孩一块玉石被继母抢去后被雷劈死玉石刘浩继母那块玉石,小孩 救 一老 乞丐 送给 小孩 一块 玉石 继母 抢去 雷劈 死 玉石 刘浩 继母 块 玉石
21659,"夜归宿舍遇陌生人洗脚 被发现后推说“走错了”洗脚,走错门",news_story,13,夜归宿舍遇陌生人洗脚被发现后推说走错了洗脚走错门,夜归 宿舍 遇 陌生人 洗脚 发现 推说 走错 洗脚 走 错门
21660,"我结婚哥哥给我十万红包，上厕所时听见他接电话，我含泪还给了他还给你,婚礼,饭店,红包",news_story,13,我结婚哥哥给我十万红包上厕所时听见他接电话我含泪还给了他还给你婚礼饭店红包,结婚 哥哥 十万 红包 厕所 时 听见 接电话 含泪 还给 还给 婚礼 饭店 红包


In [8]:
#划分训练集、测试集
X_train, X_test, y_train, y_test = train_test_split(df['cut_data'], df['id'], random_state = 0)

In [9]:
#用训练集生成词频向量和TF-IDF向量
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [10]:
#用测试集生成F-IDF向量
X_test_counts = count_vect.transform(X_test)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)

### 手写朴素贝叶斯分类器

In [11]:
#朴素贝斯函数
#train_tfidf文档词向量矩阵16246x46840
#trainlabel每条文档类别标签16246
def trainByes(train_tfidf,trainlabel):
    numTrainDocs = train_tfidf.shape[0] #训练的文档数据条数 
    numWords = train_tfidf.shape[1] #计算每条文档的词个数,这里是tfidf向量的维度
    
    #定义总共14个类别
    Types = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
    types = len(Types) #14
    
    #文档属于各个类别的概率
    pFileType = []
    for i in range(types):
        pFileType.append(trainlabel.value_counts()[i] / float(numTrainDocs))#trainlabel.value_counts()[i]对类别为i的文档进行计数
    
    #用拉普拉斯平滑,初始化所有词出现数为1，避免某一个概率值为0   
    pWord = np.ones([types, train_tfidf.shape[1]]) # 维度：(14,tfidf维度)

    #分母初始化为2,拉普拉斯平滑
    pDemo = []
    for i in range(types):
        pDemo.append(2.0)
        
    # 各个类的矩阵相加
    #y_train的index是3497/1956/2775...的形式，不是train_tfidf的维度顺序，因此每遍历一个y_train索引就对应train_tfidf矩阵行数+1
    count = -1
    for i in y_train.index:
        count += 1
        for j in Types:
            if y_train[i] == Types[j]:
                pWord[j] += train_tfidf[count]
                pDemo[j] += np.sum(train_tfidf[count])

    #计算各类条件概率组
    ##结果取对数，避免大量小数数相乘造成的影响
    pVect = []#14个元素对应每一种类别的条件概率
    for i in range(types):
        pVect.append(np.log(pWord[i]/pDemo[i]))

    return pVect,pFileType

In [12]:
pVect,pFileType = trainByes(X_train_tfidf,y_train)

In [13]:
#用朴素贝叶斯函数进行分类
def classifyBayes(test_tfidf,pVect,pFileType):
    #将pVect转为矩阵方便计算
    pVectMat = np.mat(pVect)
    
    Types = [0,1,2,3,4,5,6,7,8,9,10,11,12,13]
    types = len(Types) #14     
    
    #采用矩阵计算
    #(数据条数,文本向量维度)*(14,文本向量维度.T) = (数据条数,14)
    temp = test_tfidf * pVectMat.T#结果矩阵每行对应一个词条各个类别的概率
    for i in Types:
        temp[:,i] += np.log(pFileType[i]) 
    
    #即将矩阵每行概率最大的元素所对应的列索引作为对应的类别
    pClass = np.argmax(temp, axis=1)
            
    return pClass

In [14]:
y_pred = classifyBayes(X_test_tfidf,pVect,pFileType)

In [15]:
y_pred

matrix([[ 5],
        [10],
        [ 0],
        ...,
        [ 6],
        [13],
        [ 6]], dtype=int64)

In [16]:
y_test

8933      5
1640     10
19426     0
8050      5
5440     11
         ..
9365     11
20270     8
3577      6
4896      3
541       6
Name: id, Length: 5416, dtype: int64

In [17]:
#评估结果
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print(accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=id_df['label'].values))

0.7348596750369276
                    precision    recall  f1-score   support

  news_agriculture       0.88      0.72      0.79       343
         news_game       0.88      0.86      0.87       402
      news_culture       0.88      0.71      0.79       392
news_entertainment       0.94      0.42      0.58       425
       news_sports       0.98      0.78      0.87       468
      news_finance       0.92      0.58      0.71       409
        news_house       0.86      0.87      0.87       315
          news_car       0.95      0.81      0.87       424
          news_edu       0.89      0.77      0.83       409
         news_tech       0.83      0.77      0.80       509
     news_military       0.83      0.82      0.82       354
       news_travel       0.83      0.71      0.77       394
        news_world       0.83      0.67      0.74       399
        news_story       0.15      0.95      0.26       173

          accuracy                           0.73      5416
         macro avg 